In [1]:
# Import system modules

import os
import arcpy
from arcpy.sa import *
arcpy.env.overwriteOutput = True

# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

In [2]:
# create definitions for NDWI band math using raster calculator and reclassify to show only water

def sentinelNDWI(Band3,Band8,outImage):
    output_raster = (Band3-Band8)/(Band3+Band8)
    # output_raster = (Raster(output_raster1),[-1,0,0,1],[0, 1])                                       
    output_raster.save(outImage)

def reclassH2O(h2O_Image):
    out_water = arcpy.sa.Reclassify(in_raster=ndwiFullPath,reclass_field="VALUE", remap="-1 0 NODATA;0 1 1",missing_values="DATA")
    out_water.save(h2O_Image)

In [3]:
# Loop through the folders and get a list of SAFE directories that contain the Sentinel data
# for each directory find the Band 3 and Band 8 images

directory = r'E:\Sentinel\Tulare'
outFolder = 'F:\\ArcGIS Pro Projects VOL3\\Tulare Lake NDWI\\Data\\'
 
for folder in os.listdir(directory):
    f = os.path.join(directory, folder)
    if os.path.isdir(f):
        for root, dirs, files in os.walk(f):
            for file in files:
                if file.endswith("B03_10m.jp2"):
                    ndwiImage = str(file.split('_')[0]) + '_' + str(file.split('_')[1]) + '_NDWI.tif'
                    ndwiFullPath = outFolder + ndwiImage
                    waterImage = str(file.split('_')[0]) + '_' + str(file.split('_')[1]) + '_h2O.tif'
                    waterFullPath = outFolder + waterImage
                    B03 = Raster(os.path.join(root, file))
                if file.endswith("B08_10m.jp2"):
                    B08 = Raster(os.path.join(root, file))
                    sentinelNDWI(B03,B08,ndwiFullPath)
                    reclassH2O(waterFullPath)